In [76]:
import numpy as np
import json
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [68]:
train_data = []

with open('../data/domain1_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        train_data.append(data)

In [69]:
X = np.array([entry["text"] for entry in train_data])
y = np.array([entry["label"] for entry in train_data])

<ipython-input-69-07b0c7f8f9f5>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array([entry["text"] for entry in train_data])


In [70]:
#split training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [71]:
vectorizer = CountVectorizer(ngram_range=(1,2))
#vectorizer = TfidfVectorizer()
X_train_text = [' '.join(map(str, sample)) for sample in X_train]
X_test_text = [' '.join(map(str, sample)) for sample in X_test]

X_train_transformed = vectorizer.fit_transform(X_train_text)
X_test_transformed = vectorizer.transform(X_test_text)

In [72]:
logistic_model = LogisticRegression()

In [77]:
# with open('../models/logistic_model(N-grams).pkl', 'wb') as model_file:
#     pickle.dump(logistic_model, model_file)

In [73]:
logistic_model.fit(X_train_transformed, y_train)

LogisticRegression()

In [74]:
y_pred = logistic_model.predict(X_test_transformed)

# output acc
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.926923076923077


In [75]:
with open('../data/output_csv/logistic_output.csv', 'w') as output_file:
    output_file.write('id,class\n')  

    # read in dataset
    with open('../data/test_set.json', 'r') as file:
        for line in file:
            entry = json.loads(line)
            text = entry["text"]

            # word embedding
            X_test = vectorizer.transform([" ".join(map(str, text))])

            # use logistic model to predict
            prediction = logistic_model.predict(X_test)

            # output csv
            output_file.write(f"{entry['id']},{prediction[0]}\n")